# 💾 Acceso a Azure Data Lake Storage (ADLS Gen2) desde Databricks

Este diagrama ilustra cómo **Databricks** se conecta y accede a los datos almacenados en **Azure Data Lake Storage Gen2 (ADLS Gen2)**.

---

## 💻 Componentes Principales

* **Databricks:** La plataforma de análisis.
    * **Cluster:** El recurso computacional que ejecuta las tareas.
    * **Notebook:** El entorno donde escribes y ejecutas el código (por ejemplo, Python o SQL).
* **Azure:** El proveedor de la nube.
    * **Storage Account:** El contenedor que aloja los datos.
    * **ADLS Gen2:** La capa de almacenamiento optimizada para Big Data.

---

## 🔑 Métodos de Autenticación y Seguridad

Para que el Cluster/Notebook de Databricks pueda leer o escribir en ADLS Gen2, necesita autenticarse. Los métodos de autenticación comunes son:

### 1. Service Principal (Microsoft Entra ID/Azure Active Directory) 🛡️

* Es el método **más recomendado** y seguro para entornos de producción.
* Se utiliza una identidad de aplicación (el **Service Principal**) registrada en **Microsoft Entra ID** (anteriormente Azure AD) para obtener acceso.

### 2. Storage Access Keys 🔑

* Claves maestras que otorgan **acceso completo** a la cuenta de almacenamiento.
* **No son recomendadas** para acceso de producción debido a que si se filtran, comprometen toda la cuenta.

### 3. Shared Access Signature (SAS Token) 🔗

* Tokens temporales que otorgan acceso **limitado** (por tiempo, permisos y recursos específicos) a los datos.

